In [1]:
# coding=utf-8
"""
Credit by Liao Jialing

"""
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

# 读入用户基本信息进行数据清洗，做one-hot处理等

In [2]:
dataset = pd.read_csv('E:/user_all.csv')

D:\Users\Jialing Liao\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dataset['member_age'] = 2017 - dataset['member_birthday'][dataset['member_birthday'].notnull()]

In [4]:
dataset['song_age'] = 2017 - dataset['year'][dataset['year'].notnull()]

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503879 entries, 0 to 503878
Columns: 114 entries, customerid to song_age
dtypes: float64(105), object(9)
memory usage: 442.1+ MB


In [6]:
len(pd.value_counts(dataset['member_age']))

67

In [7]:
dataset['member_age'].notnull().sum()

15559

In [8]:
len(pd.value_counts(dataset['song_age']))

41

In [9]:
dataset['song_age'].notnull().sum()

2300

## 填充缺失值

In [10]:
for i in dataset.columns:
    if dataset.shape[0] != dataset[i].notnull().sum():
        print(i)

registertime
customer_status
openid
wechat_sex
wechat_city
wechat_province
wechat_country
kmid
member_birthday
km_sex
accountcash
accountbalance
integralbalance
consumetotal
consumetimes
lastconsume_diff_create_day
lastconsume_diff_create_hour
lastconsume_diff_now
coupon_get_cnt
coupon_get_sum
source_company_id
source_company_cnt
get_coupon_type_cnt
get_com_roomnum
get_com_regioncode
get_com_managetype
get_com_vodsystemtype
get_coupon_crosscom
use_coupon_sum
use_coupon_order_sum
use_coupon_type_cnt
use_company_id
use_company_cnt
des_cnt
des_charge_sum
des_procharge_sum
des_reduce_sum
des_realcharge_sum
des_workday_cnt
des_weekend_cnt
des_work_realchargesum
des_weekend_realchargesum
des_charge_mean
des_realcharge_mean
des_guestnumber_mean
des_guestnumber_max
des_guestnumber_min
des_charge_max
des_charge_min
des_realcharge_max
des_realcharge_min
use_time_sum
des_beforedawn_cnt
des_morning_cnt
des_afternoon_cnt
des_night_cnt
des_use_beforedawn_cnt
des_use_morning_cnt
des_use_afternoon_cnt

In [11]:
dataset.drop('registertime',axis=1,inplace=True)

In [12]:
dataset.shape

(503879, 113)

In [13]:
dataset.drop_duplicates('customerid').shape

(503862, 113)

### customerid去重（对数据量几乎无影响）

In [14]:
dataset.drop_duplicates('customerid',inplace=True)

In [15]:
# for col in dataset.columns.values:
#     print(col, dataset[col].notnull().sum())

In [16]:
dataset.shape

(503862, 113)

### customerid唯一且非空，可drop掉总表的kmid和openid

In [17]:
dataset.drop(['kmid','openid'],axis=1,inplace=True)

### customer_status无区别，无用，弃之

In [18]:
dataset.drop('customer_status',axis=1,inplace=True)

In [19]:
dataset.shape

(503862, 110)

### wechat_city wechat_province wechat_country

In [20]:
for col in ['wechat_city','wechat_province','wechat_country']:
    dataset[col].fillna('missing',inplace=True)

In [21]:
dataset.shape

(503862, 110)

In [22]:
dataset.head()

,customerid,wechat_sex,wechat_city,wechat_province,wechat_country,member_birthday,km_sex,accountcash,accountbalance,integralbalance,...,shop_companyid,shop_com_cnt,shop_order_cnt,shop_cross_com,shop_com_roomnum,shop_com_regioncode,shop_com_managetype,shop_com_vodsystemtype,member_age,song_age
0,5792237,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5792238,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5792239,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5792241,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5792242,NaN,missing,missing,missing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### km_sex:1-男性，2-女性，0-未知

In [23]:
pd.value_counts(dataset['km_sex'])

1    8607
2    6311
0     641
Name: km_sex, dtype: int64

### wechat_sex和km_sex合并为gender：1-男性，2-女性，0-未知（缺失值填0）

In [24]:
dataset['wechat_sex'][dataset['wechat_sex'].isnull()]=dataset['km_sex']

D:\Users\Jialing Liao\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
pd.value_counts(dataset['wechat_sex'])

0.0          51480
0            23393
1.0          11019
2.0           8472
1             1631
2             1291
-1               2
dao              1
pan              1
慧子               1
Z\               1
距离令人伤了神。         1
Monaco           1
a promise        1
  mj             1
Name: wechat_sex, dtype: int64

In [26]:
dataset['gender'] = dataset['wechat_sex']
dataset.drop(['wechat_sex','km_sex'],axis=1,inplace=True)

In [27]:
dataset['gender'].fillna(0,inplace=True)

In [28]:
def sex_encode(data):
    if data==1:
        return 1
    elif data==2:
        return 2
    else:
        return 0

#### 清除脏数据

In [29]:
dataset['gender'] = map(sex_encode,dataset['gender'])

In [30]:
pd.value_counts(dataset['gender'])

0    484371
1     11019
2      8472
Name: gender, dtype: int64

### 转换为0-男性，1-女性，2-未知（缺失值为2）

In [31]:
def transfer_gender(data):
    if data==0:
        return 2
    elif data==1:
        return 0
    else:
        return 1

In [32]:
dataset['gender'] = map(transfer_gender,dataset['gender'])

In [33]:
pd.value_counts(dataset['gender'])

2    484371
0     11019
1      8472
Name: gender, dtype: int64

In [34]:
dataset.shape

(503862, 109)

# checkpoint 1,暂存csv

In [35]:
# dataset.to_csv('E:/dataset_sex.csv',index=False)

#### wechat city粒度太细，数据太稀疏直接去掉

In [36]:
dataset.drop('wechat_city',axis=1,inplace=True)

#### wechat_province

In [37]:
dataset['wechat_province'].notnull().sum() # 填充前的值

503862

In [38]:
def province_encode(data):
    province_set = ['辽宁','吉','黑龙江','河北','山西','陕西','山东','安徽','江苏','浙江','河南','湖北','湖南','江西',
                    '台湾','福建','云南','海南','四川','贵州','广东','甘肃','青海','西藏','新疆','广西','内蒙古','宁夏',
                    '北京','天津','上海','重庆']
    if data in province_set:
        return data
    elif data == 'null' or data=='missing':
        return '未知'
    else:
        return '国外'

In [39]:
dataset['wechat_province'] = map(province_encode,dataset['wechat_province'])

In [40]:
pd.value_counts(dataset['wechat_province'])

未知     496867
广东       2736
上海       1070
重庆        997
国外        681
福建        558
四川        122
湖南        108
安徽         99
湖北         74
江苏         71
浙江         66
江西         50
广西         47
云南         46
山东         37
河南         35
贵州         34
北京         30
辽宁         21
海南         18
陕西         16
黑龙江        14
河北         13
甘肃         12
内蒙古        10
新疆         10
山西          8
天津          4
西藏          4
台湾          2
青海          2
Name: wechat_province, dtype: int64

In [41]:
# get_dummies
wechat_province = pd.get_dummies(dataset.wechat_province)
wechat_province = wechat_province.rename(columns=lambda x:'wechat_province_'+str(x))

In [42]:
dataset = pd.concat([dataset,wechat_province],axis=1)
dataset.drop('wechat_province',axis=1,inplace=True)

In [43]:
wechat_province.head()

,wechat_province_上海,wechat_province_云南,wechat_province_内蒙古,wechat_province_北京,wechat_province_台湾,wechat_province_四川,wechat_province_国外,wechat_province_天津,wechat_province_安徽,wechat_province_山东,...,wechat_province_湖南,wechat_province_甘肃,wechat_province_福建,wechat_province_西藏,wechat_province_贵州,wechat_province_辽宁,wechat_province_重庆,wechat_province_陕西,wechat_province_青海,wechat_province_黑龙江
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
dataset.shape

(503862, 139)

In [45]:
dataset.head()

,customerid,wechat_country,member_birthday,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,...,wechat_province_湖南,wechat_province_甘肃,wechat_province_福建,wechat_province_西藏,wechat_province_贵州,wechat_province_辽宁,wechat_province_重庆,wechat_province_陕西,wechat_province_青海,wechat_province_黑龙江
0,5792237,missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,5792238,missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,5792239,missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,5792241,missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5792242,missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


#### wechat_country

In [46]:
dataset['wechat_country'].notnull().sum() # 填充前的值

503862

In [47]:
pd.value_counts(dataset['wechat_country'])

missing     422443
null         73783
中国            6353
-1             175
韩国              78
冰岛              65
安道尔             57
澳大利亚            49
美国              45
法国              39
中国香港            37
阿尔巴尼亚           32
爱尔兰             31
阿拉伯联合酋长国        27
百慕大             27
日本              27
泽西岛             26
阿尔及利亚           21
阿鲁巴             18
英国              18
瑞士              17
马尔代夫            17
毛里求斯            16
奥地利             15
希腊              15
不丹              13
加拿大             13
巴巴多斯岛           12
智利              12
意大利             12
             ...  
斯洛伐克             1
利比里亚             1
巴西               1
斯洛文尼亚            1
沙特阿拉伯            1
克罗地亚             1
哥斯达黎加            1
格林纳达             1
博茨瓦纳             1
刚果民主共和国          1
苏里南              1
云南               1
阿塞拜疆             1
帕劳群岛             1
蒙古               1
汤加               1
纳米比亚             1
委内瑞拉             1
塔吉克斯坦            1
葡萄牙              1
所罗门群岛            1
匈牙利         

In [48]:
def encode_country(data):
    if data == '中国' or data == '中国香港':
        return '中国'
    elif data == 'missing' or data =='null':
        return '未知'
    else:
        return '国外'

In [49]:
dataset['wechat_country'] = map(encode_country, dataset['wechat_country'])

In [50]:
pd.value_counts(dataset['wechat_country'])

未知    496226
中国      6390
国外      1246
Name: wechat_country, dtype: int64

In [51]:
wechat_country = pd.get_dummies(dataset.wechat_country)
wechat_country = wechat_country.rename(columns=lambda x:'wechat_country_'+str(x))
dataset = pd.concat([dataset,wechat_country],axis=1)
dataset.drop('wechat_country',axis=1,inplace=True)

In [52]:
dataset.shape

(503862, 141)

In [53]:
dataset.head()

,customerid,member_birthday,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now,...,wechat_province_西藏,wechat_province_贵州,wechat_province_辽宁,wechat_province_重庆,wechat_province_陕西,wechat_province_青海,wechat_province_黑龙江,wechat_country_中国,wechat_country_国外,wechat_country_未知
0,5792237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
1,5792238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
2,5792239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
3,5792241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
4,5792242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


# checkpoint 2

In [54]:
# dataset.to_csv('E:/dataset_province_country.csv',index=False)

### member_birthday

In [55]:
dataset.drop('member_birthday',axis=1,inplace=True)

In [56]:
dataset.shape

(503862, 140)

### accountcash accountbalance integralbalance consumetotal consumetimes

In [57]:
for col in ['accountcash', 'accountbalance', 'integralbalance', 'consumetotal', 'consumetimes']:
    print(col,dataset[col].dtypes)

('accountcash', dtype('float64'))
('accountbalance', dtype('float64'))
('integralbalance', dtype('float64'))
('consumetotal', dtype('float64'))
('consumetimes', dtype('float64'))


In [58]:
for col in ['accountcash', 'accountbalance', 'integralbalance', 'consumetotal', 'consumetimes']:
    dataset[col].fillna(0,inplace=True)

In [59]:
dataset.head()

,customerid,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now,coupon_get_cnt,...,wechat_province_西藏,wechat_province_贵州,wechat_province_辽宁,wechat_province_重庆,wechat_province_陕西,wechat_province_青海,wechat_province_黑龙江,wechat_country_中国,wechat_country_国外,wechat_country_未知
0,5792237,0,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
1,5792238,0,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
2,5792239,0,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
3,5792241,0,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
4,5792242,0,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1


### ？？？lastconsume_diff_create_day，lastconsume_diff_create_hour，lastconsume_diff_now

In [60]:
def clear_timedelta(data):
    if data<0:
        return 0
    else:
        return data

In [61]:
dataset['lastconsume_diff_create_day'] = map(clear_timedelta,dataset['lastconsume_diff_create_day'])

In [62]:
dataset['lastconsume_diff_now'] = map(clear_timedelta,dataset['lastconsume_diff_now'])

### 这里填0真的好吗？？？？

In [63]:
for col in ['lastconsume_diff_create_day','lastconsume_diff_create_hour','lastconsume_diff_now']:
    dataset[col].fillna(0,inplace=True)

# checkpoint 3

In [64]:
# dataset.to_csv('E:/dataset_3.csv',index=False)

### ？？？coupon_get_cnt coupon_get_sum source_company_cnt get_coupon_type_cnt get_com_roomnum get_coupon_crosscom 

In [65]:
for col in ['coupon_get_cnt','coupon_get_sum','source_company_cnt','get_coupon_type_cnt','get_com_roomnum','get_coupon_crosscom']:
    dataset[col].fillna(0,inplace=True)

In [66]:
dataset.shape

(503862, 140)

In [67]:
# for col in ['coupon_get_cnt','coupon_get_sum','source_company_cnt','get_coupon_type_cnt','get_com_roomnum',
#             'get_com_roomnum','get_coupon_crosscom']:
#     print(col,pd.value_counts(dataset[col]))

### ？？？use_coupon_sum  use_coupon_order_sum  use_coupon_type_cnt  use_company_cnt

In [68]:
for col in ['use_coupon_sum','use_coupon_order_sum','use_coupon_type_cnt','use_company_cnt']:
    dataset[col].fillna(0,inplace=True)

In [69]:
dataset.shape

(503862, 140)

### source_company_id use_company_id填-1 后续怎么处理？？？

In [70]:
for col in ['source_company_id','use_company_id']:
    dataset[col].fillna(-1,inplace=True)

### get_com_regioncode get_com_managetype get_com_vodsystemtype

In [71]:
pd.value_counts(dataset['get_com_regioncode'])

440000    54870
310000    22785
500000    18474
350000     5198
Name: get_com_regioncode, dtype: int64

In [72]:
pd.value_counts(dataset['get_com_managetype'])

1    101038
2       289
Name: get_com_managetype, dtype: int64

In [73]:
pd.value_counts(dataset['get_com_vodsystemtype'])

0    101327
Name: get_com_vodsystemtype, dtype: int64

In [74]:
dataset.drop('get_com_vodsystemtype',axis=1,inplace=True)

In [75]:
for col in ['get_com_regioncode','get_com_managetype']:
    dataset[col].fillna('not_known',inplace=True)

In [76]:
dataset.shape

(503862, 139)

In [77]:
get_regioncode = pd.get_dummies(dataset.get_com_regioncode)
get_regioncode = get_regioncode.rename(columns=lambda x:'get_com_regioncode_'+str(x))
dataset = pd.concat([dataset,get_regioncode],axis=1)
dataset.drop('get_com_regioncode',axis=1,inplace=True)

In [78]:
dataset.shape

(503862, 143)

In [79]:
get_com_managetype = pd.get_dummies(dataset.get_com_managetype)
get_com_managetype = get_com_managetype.rename(columns=lambda x:'get_com_managetype_'+str(x))
dataset = pd.concat([dataset,get_com_managetype],axis=1)
dataset.drop('get_com_managetype',axis=1,inplace=True)

In [80]:
dataset.shape

(503862, 145)

### des表大部分连续数据

In [81]:
for col in ['des_cnt','des_charge_sum','des_procharge_sum','des_reduce_sum','des_realcharge_sum','des_workday_cnt',
            'des_weekend_cnt','des_work_realchargesum','des_weekend_realchargesum','des_charge_mean','des_realcharge_mean',
            'des_guestnumber_mean','des_guestnumber_max','des_guestnumber_min','des_charge_max','des_charge_min',
            'des_realcharge_max','des_realcharge_min','use_time_sum','des_beforedawn_cnt','des_morning_cnt',
            'des_afternoon_cnt','des_night_cnt','des_use_beforedawn_cnt','des_use_morning_cnt','des_use_afternoon_cnt',
            'des_use_night_cnt','des_use_beforedawn_realchargesum','des_use_morning_realchargesum',
            'des_use_afternoon_realchargesum','des_use_night_realchargesum','des_company_cnt','des_com_roomnum','des_large_cnt',
            'des_medium_cnt','des_small_cnt','des_reduce_cnt']:
    dataset[col].fillna(0,inplace=True)

In [82]:
dataset.shape

(503862, 145)

### ???des_use_reg 每个用户平均预订包厢和使用包厢的天数差

In [83]:
def encode_negative(data):
    if data<0:
        return 0
    else:
        return data

In [84]:
dataset['des_use_reg'] = map(encode_negative,dataset['des_use_reg'])

In [85]:
dataset['des_use_reg'].fillna(0,inplace=True)

In [86]:
pd.value_counts(dataset['des_use_reg'])

0.000000    503728
1.000000        51
2.000000        22
0.500000        11
0.333333         8
3.000000         6
0.250000         4
1.500000         4
0.750000         3
5.000000         3
4.000000         2
0.200000         2
0.428571         2
0.166667         2
0.666667         2
5.750000         1
0.800000         1
3.200000         1
1.333333         1
0.096774         1
0.111111         1
6.000000         1
0.161191         1
1.666667         1
2.333333         1
0.548387         1
2.250000         1
Name: des_use_reg, dtype: int64

### ???des_companyid

In [87]:
dataset['des_companyid'].fillna(-1,inplace=True)

In [88]:
pd.value_counts(dataset['des_companyid'])

-1        502689
 4018        227
 5677        130
 6131        114
 6130         47
 1727         44
 8860         36
 4800         34
 5624         32
 180          31
 2131         26
 7971         26
 4831         25
 8005         23
 5401         18
 10093        18
 7688         15
 4148         14
 5046         11
 7121         10
 10629         9
 3753          9
 3868          9
 3938          9
 640           8
 8879          8
 2413          8
 3705          8
 6453          8
 4202          7
           ...  
 2726          1
 2748          1
 8160          1
 8158          1
 8144          1
 7598          1
 7479          1
 7030          1
 6987          1
 6500          1
 6479          1
 6307          1
 6282          1
 6227          1
 9202          1
 5833          1
 10180         1
 5608          1
 5109          1
 5021          1
 4938          1
 11291         1
 4034          1
 10235         1
 3954          1
 3919          1
 3824          1
 3239         

In [89]:
len(pd.value_counts(dataset['des_companyid']))

117

### des_com_managetype des_com_vodsystemtype des_com_regioncode数据太少，全部drop

In [90]:
pd.value_counts(dataset['des_com_vodsystemtype'])

0    238
2      2
Name: des_com_vodsystemtype, dtype: int64

In [91]:
pd.value_counts(dataset['des_com_managetype'])

1    240
Name: des_com_managetype, dtype: int64

In [92]:
pd.value_counts(dataset['des_com_regioncode'])

440000    105
500000     61
350000     54
310000     20
Name: des_com_regioncode, dtype: int64

In [93]:
dataset.drop(['des_com_vodsystemtype','des_com_managetype','des_com_regioncode'],axis=1,inplace=True)

In [94]:
dataset.shape

(503862, 142)

# checkpoint 4

In [95]:
# dataset.to_csv('E:/dataset4.csv',index=False)

### song表的部分

#### 不需要songid了

In [96]:
dataset.drop('songid',axis=1,inplace=True)

In [97]:
for col in ['age_year','age_lan']:
    dataset[col].fillna('not_known',inplace=True)

In [98]:
pd.value_counts(dataset['age_year'])

not_known    501562
5.0            2013
4.0             172
3.0              79
2.0              36
Name: age_year, dtype: int64

In [99]:
pd.value_counts(dataset['age_lan'])

not_known    497245
2.0            5672
1.0             543
3.0             402
Name: age_lan, dtype: int64

In [100]:
age_year = pd.get_dummies(dataset.age_year)
age_year = age_year.rename(columns=lambda x:'age_from_song_year_cut_'+str(x))
dataset = pd.concat([dataset,age_year],axis=1)
dataset.drop('age_year',axis=1,inplace=True)

In [101]:
age_lan = pd.get_dummies(dataset.age_lan)
age_lan = age_lan.rename(columns=lambda x:'age_from_song_language_'+str(x))
dataset = pd.concat([dataset,age_lan],axis=1)
dataset.drop('age_lan',axis=1,inplace=True)

In [102]:
dataset.drop('year',axis=1,inplace=True)

In [103]:
dataset.shape

(503862, 147)

In [104]:
dataset['song_age'].fillna(999,inplace=True)

### song表最后一项和shop表大部分连续数据

In [105]:
for col in ['song_play_cnt','shop_real_sum','shop_original_sum','shop_wine_cnt','shop_cigarette_cnt','shop_cnt',
'shop_before_dawn_cnt','shop_morning_cnt','shop_afternoon_cnt','shop_night_cnt','shop_weekday_cnt',
'shop_weekend_cnt','shop_before_dawn_sum','shop_morning_sum','shop_afternoon_sum','shop_night_sum',
'shop_weekday_sum','shop_weekend_sum','shop_real_sum_avg','shop_original_sum_avg','shop_real_sum_max',
'shop_original_sum_max','shop_real_sum_min','shop_original_sum_min','shop_com_cnt','shop_order_cnt',
'shop_cross_com','shop_com_roomnum']:
    dataset[col].fillna(0,inplace=True)

In [106]:
dataset.shape

(503862, 147)

In [107]:
dataset['shop_companyid'].fillna(-1,inplace=True)

### ???shop_com_regioncode shop_com_managetype shop_com_vodsystemtype

In [108]:
pd.value_counts(dataset['shop_com_regioncode'])

440000    27324
310000    10559
500000     8572
350000     3507
Name: shop_com_regioncode, dtype: int64

In [109]:
pd.value_counts(dataset['shop_com_managetype'])

1    49812
2      150
Name: shop_com_managetype, dtype: int64

In [110]:
pd.value_counts(dataset['shop_com_vodsystemtype'])

0    49962
Name: shop_com_vodsystemtype, dtype: int64

In [111]:
dataset.drop('shop_com_vodsystemtype',axis=1,inplace=True)

In [112]:
dataset.shape

(503862, 146)

In [113]:
for col in ['shop_com_regioncode','shop_com_managetype']:
    dataset[col].fillna('not_known',inplace=True)

In [114]:
shop_com_regioncode = pd.get_dummies(dataset.shop_com_regioncode)
shop_com_regioncode = shop_com_regioncode.rename(columns=lambda x:'shop_com_regioncode_'+str(x))
dataset = pd.concat([dataset,shop_com_regioncode],axis=1)
dataset.drop('shop_com_regioncode',axis=1,inplace=True)

In [115]:
shop_com_managetype = pd.get_dummies(dataset.shop_com_managetype)
shop_com_managetype = shop_com_managetype.rename(columns=lambda x:'shop_com_managetype_'+str(x))
dataset = pd.concat([dataset,shop_com_managetype],axis=1)
dataset.drop('shop_com_managetype',axis=1,inplace=True)

In [116]:
dataset.shape

(503862, 152)

In [117]:
shop_com_managetype.shape

(503862, 3)

In [118]:
shop_com_regioncode.shape

(503862, 5)

In [119]:
dataset['member_age'].fillna(0,inplace=True)

# 暂存用户数据清理后的表，连续值没有处理

In [120]:
dataset.to_csv('E:/user_all_cleared.csv',index=False)

# 查看连续值和离散值个数

In [121]:
sequential_cnt = 0
discrete_cnt = 0
print('--------Sequential:--------')
for col in dataset.columns:
    if len(pd.value_counts(dataset[col])) > 10:
        sequential_cnt += 1
        print(col)
    else:
        discrete_cnt += 1
print('------------------------------')
print('sequential_cnt:',sequential_cnt)
print('discrete_cnt:',discrete_cnt)
print('--------Discrete:----------')
for col in dataset.columns:
    if len(pd.value_counts(dataset[col])) < 10:
        print(col)

--------Sequential:--------
customerid
accountcash
accountbalance
integralbalance
consumetotal
consumetimes
lastconsume_diff_create_day
lastconsume_diff_create_hour
lastconsume_diff_now
coupon_get_cnt
coupon_get_sum
source_company_id
source_company_cnt
get_coupon_type_cnt
get_com_roomnum
use_coupon_sum
use_coupon_order_sum
use_company_id
use_company_cnt
des_cnt
des_charge_sum
des_procharge_sum
des_reduce_sum
des_realcharge_sum
des_workday_cnt
des_work_realchargesum
des_weekend_realchargesum
des_charge_mean
des_realcharge_mean
des_guestnumber_mean
des_guestnumber_max
des_guestnumber_min
des_charge_max
des_charge_min
des_realcharge_max
des_realcharge_min
use_time_sum
des_afternoon_cnt
des_use_night_cnt
des_use_beforedawn_realchargesum
des_use_morning_realchargesum
des_use_afternoon_realchargesum
des_use_night_realchargesum
des_use_reg
des_companyid
des_company_cnt
des_com_roomnum
song_play_cnt
shop_real_sum
shop_original_sum
shop_cnt
shop_before_dawn_cnt
shop_morning_cnt
shop_afternoon_c